# Pendahuluan

In [1]:
# Library yang dibutuhkan

# Pertama-tama, import terlebih dahulu library yang dibutuhkan, antara lain secara berurutan adalah :
# json,
# numpy,
# pandas, dan
# requests.

# Library numpy dan pandas diimport sebagai aliasnya.

import json
import numpy as np
import pandas as pd
import requests
from pandas import json_normalize

# Mengakses Data Covid-19 dari https://covid19-api.org

In [2]:
# Membuat Fungsi get API

# Buat fungsi python get_json dengan parameter api_url. fungsi ini akan mengembalikan value berupa python dictionary. 
# Jika status_code yang dihasilkan adalah 200. Jika tidak, maka value yang dikembalikan adalah None.

def get_json(api_url):
 response = requests.get(api_url)
 if response.status_code == 200:
  return json.loads(response.content.decode('utf-8'))
 else:
  return None

In [3]:
# Memanggil API Covid-19

# Rekapitulasi data COVID-19 global berada di https://covid19-api.org/. 
# Gunakan parameter record_date untuk mengambil data covid19 di seluruh negara pada tanggal tertentu. 
# Untuk kasus ini, gunakan record_date '2020-08-17', dan masukkan hasil respon api ke variable df_covid_worldwide.

# Untuk mendapatkan dataframe covid19, gunakan fungsi pd.io.json.json_normalize dan panggil function yang sudah dibuat sebelumnya, 
# yaitu get_json() Print sample data covid dengan menggunakan head() function.

record_date = '2020-08-17'
covid_url = 'https://covid19-api.org/api/status?date='+record_date
df_covid_worldwide = pd.json_normalize(get_json(covid_url))

print(df_covid_worldwide.head())

  country          last_update    cases  deaths  recovered
0      US  2020-08-16T23:27:50  5401167  170019    1833067
1      BR  2020-08-16T23:27:50  3340197  107852    2655017
2      IN  2020-08-16T23:27:50  2589682   49980    1862258
3      RU  2020-08-16T23:27:50   920719   15653     731444
4      ZA  2020-08-16T23:27:50   587345   11839     472377


In [4]:
# Merubah Format date

# Ubah format kolom 'last_update' menggunakan fungsi to_datetime dengan format YYYY-mm-dd HH:MM:SS. 
# Lalu ubah bentuk datetime ke bentuk date dengan fungsi date() melalui fungsi lambda untuk mengubah setiap row-nya.

df_covid_worldwide['last_update'] = pd.to_datetime(df_covid_worldwide['last_update'], format='%Y-%m-%d %H:%M:%S')
df_covid_worldwide['last_update'] = df_covid_worldwide['last_update'].apply(lambda x: x.date())

In [7]:
# Mengambil Data Countries

# Buatlah dataframe countries dengan memanggil variable countries_url ke api covid19-api.org. 
# Lalu ubah nama kolom alpha2 menjadi country, kemudian ambil hanya kolom name dan country saja untuk dataframe countries (df_countries) ini.

# Akhirnya cetaklah lima data teratas dari dataframe countries (df_countries).

countries_url = 'https://covid19-api.org/api/countries'
df_countries = pd.json_normalize(get_json(countries_url))
df_countries = df_countries.rename(columns={'alpha2': 'country'})[['name','country']]
print(df_countries.head())

          name country
0  Afghanistan      AF
1      Albania      AL
2      Algeria      DZ
3      Andorra      AD
4       Angola      AO


# Kasus Covid-19 di Dunia dan Negara Mana yang Tertinggi

In [ ]:
# Merge Covid-19 Data dan Countries

# Selanjutnya adalah mapping data covid19 dan data country. 
# Gunakan fungsi merge pada pandas untuk menggabungkan df_covid_worldwide dan df_countries. 
# Untuk merge, gunakan kolom country. Lalu print sample data dengan menggunakan head().

df_covid_denormalized = pd.merge(df_covid_worldwide, df_countries, on='country')
print(df_covid_denormalized.head())